# Data preprocessing
In this jupyter notebook, the following would be done:
1. Load the dataset
2. Get the dataset info, and see if there are any empty cells
3. Encode any categorical features into numbers

In [1]:
import pandas as pd
import os
import copy

### Load the dataset

In [2]:
#Get the data path
data_path = os.path.join(os.getcwd(), os.pardir, "data", "credit_card_transactions.csv")

#Lets check that the path is working
print(os.path.exists(data_path))

True


In [3]:
#Obtain the dataset
df_data = pd.read_csv(data_path)

In [4]:
print(df_data.columns)

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud', 'merch_zipcode'],
      dtype='object')


### Preprocessing_1: Lets set index properly
The "Unnamed:0" column refers to the index per row when the data was collected. Lets drop that one

In [5]:
#Play this cell to obtain the new 
df_preprocessed_1 = df_data.drop("Unnamed: 0", axis=1)

#Lets see the columns now.
print(df_preprocessed_1.columns)

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud', 'merch_zipcode'],
      dtype='object')


### Lets see the info of each of the features.

In [6]:
df_preprocessed_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   trans_date_trans_time  1296675 non-null  object 
 1   cc_num                 1296675 non-null  int64  
 2   merchant               1296675 non-null  object 
 3   category               1296675 non-null  object 
 4   amt                    1296675 non-null  float64
 5   first                  1296675 non-null  object 
 6   last                   1296675 non-null  object 
 7   gender                 1296675 non-null  object 
 8   street                 1296675 non-null  object 
 9   city                   1296675 non-null  object 
 10  state                  1296675 non-null  object 
 11  zip                    1296675 non-null  int64  
 12  lat                    1296675 non-null  float64
 13  long                   1296675 non-null  float64
 14  city_pop          

### Lets check for any null missing values
Before we even attempt to perform any preprocessing of the features; we first must check for null values in the data.

In [7]:
df_preprocessed_1.isnull().any()

trans_date_trans_time    False
cc_num                   False
merchant                 False
category                 False
amt                      False
first                    False
last                     False
gender                   False
street                   False
city                     False
state                    False
zip                      False
lat                      False
long                     False
city_pop                 False
job                      False
dob                      False
trans_num                False
unix_time                False
merch_lat                False
merch_long               False
is_fraud                 False
merch_zipcode             True
dtype: bool

In [8]:
for col in df_preprocessed_1.columns:
    if df_preprocessed_1[col].isnull().any():
        print(f"{col}: has missing values")

merch_zipcode: has missing values


In [9]:
print(df_preprocessed_1["merch_zipcode"].median())
print(df_preprocessed_1["merch_zipcode"].head(5))

45860.0
0    28705.0
1        NaN
2    83236.0
3        NaN
4    22844.0
Name: merch_zipcode, dtype: float64


### Zip code has missing, lets fill with median
Since zip code has missing nan values, lets actually fill them now with median

In [10]:
df_preprocessed_1["merch_zipcode"] = df_preprocessed_1['merch_zipcode'].fillna(df_preprocessed_1['merch_zipcode'].median())

In [11]:
df_preprocessed_1.isnull().any()

trans_date_trans_time    False
cc_num                   False
merchant                 False
category                 False
amt                      False
first                    False
last                     False
gender                   False
street                   False
city                     False
state                    False
zip                      False
lat                      False
long                     False
city_pop                 False
job                      False
dob                      False
trans_num                False
unix_time                False
merch_lat                False
merch_long               False
is_fraud                 False
merch_zipcode            False
dtype: bool

We know that the transaction date is pretty unique for every single row of data. However, the rest of the object features do have unique values that can be found which can be discretized properly using a labelling approach. lets do the following then
1. Lets look at init values of each to see what type of variables we have.

In [12]:
# Lest select the dataframe with only object
df_object = df_preprocessed_1.select_dtypes(exclude=["float", "int"])

# Lets loop and print just a few to see examples
for c in df_object.columns:
    print(f"{c}: {df_preprocessed_1[c].unique().shape}")

trans_date_trans_time: (1274791,)
merchant: (693,)
category: (14,)
first: (352,)
last: (481,)
gender: (2,)
street: (983,)
city: (894,)
state: (51,)
job: (494,)
dob: (968,)
trans_num: (1296675,)


### Another unique identifier! transaction number.

It seems this has too much unique values. We need data that gives us patterns wether is fraud or not. Lets check what kind of data this column and weather it stores any information.

In [13]:
df_preprocessed_1["trans_num"].head(4)

0    0b242abb623afc578575680df30655b9
1    1f76529f8574734946361c461b024d99
2    a1a22d70485983eac12b5b88dad1cf95
3    6b849c168bdad6f867558c3793159a81
Name: trans_num, dtype: object

### Drop column
This is a specific ID of the transaction. Inherently; this is doesnt contain information regarding the transaction information. <br>

This is a problem since ML models rely on these features; this is why we dropped "unnamed:0" before. Therefore, we would drop this column too.

In [14]:
#Play this cell to obtain the new 
df_preprocessed_1 = df_preprocessed_1.drop("trans_num", axis=1)

In [15]:
# Lest select the dataframe with only object
df_object = df_preprocessed_1.select_dtypes(exclude=["float", "int"])

# Lets loop and print just a few to see examples
for c in df_object.columns:
    print(f"{c}: {df_preprocessed_1[c].unique().shape}")

trans_date_trans_time: (1274791,)
merchant: (693,)
category: (14,)
first: (352,)
last: (481,)
gender: (2,)
street: (983,)
city: (894,)
state: (51,)
job: (494,)
dob: (968,)


### Lets list each of the variables.
The features we are seeing that are not numerical (considered objects); are the following:
1. trans_date_time -> datetime value
2. merchant -> categorical feature (693 unique values)
3. category -> categorical feature (14 unique values)
4. first -> categorical feature (352 unique values)
5. last -> categorical feature (481 unique values)
6. Gender -> Binary data
7. Street -> Categorical features (983 unique values)
8. City -> Categorical features (894 unique values)
9. State -> Categorical features (51 unique values)
10. job -> Categorical features (494 unique values)
11. dob -> datetime value (date of birth)

### Preprocessing_2: Lets process first the datetime variables!

These are trans_date_time and dob. We are gonna do the following:
1. Convert them into datetime types
2. Obtain the relevant information (year, month, day, minute, seconds) for trans_date_time
3. Obtain the relevant information (year, month, day) for the date of birth.

We will perform all of the pre-processing in our df_preprocessed

In [16]:
#Lets now do df_preprocessed_2
df_preprocessed_2 = copy.copy(df_preprocessed_1)

#Lets convert each of these into datetime
df_preprocessed_2["trans_date_trans_time"] = pd.to_datetime(df_preprocessed_2["trans_date_trans_time"])
df_preprocessed_2["dob"] = pd.to_datetime(df_preprocessed_2["dob"])

In [17]:
print(df_preprocessed_2["trans_date_trans_time"].head(4))

0   2019-01-01 00:00:18
1   2019-01-01 00:00:44
2   2019-01-01 00:00:51
3   2019-01-01 00:01:16
Name: trans_date_trans_time, dtype: datetime64[ns]


In [18]:
print(df_preprocessed_2["dob"].head(4))

0   1988-03-09
1   1978-06-21
2   1962-01-19
3   1967-01-12
Name: dob, dtype: datetime64[ns]


In [19]:
#Lets obtain the relevant information for the transaction datetime
df_preprocessed_2["transaction_year"] = df_preprocessed_2["trans_date_trans_time"].dt.year
df_preprocessed_2["transaction_month"] = df_preprocessed_2["trans_date_trans_time"].dt.month
df_preprocessed_2["transaction_day"] = df_preprocessed_2["trans_date_trans_time"].dt.day
df_preprocessed_2["transaction_hour"] = df_preprocessed_2["trans_date_trans_time"].dt.hour
df_preprocessed_2["transaction_minute"] = df_preprocessed_2["trans_date_trans_time"].dt.minute
df_preprocessed_2["transaction_second"] = df_preprocessed_2["trans_date_trans_time"].dt.second

In [20]:
#Lets obtain the relevant information for the date of birth
df_preprocessed_2["birth_year"] = df_preprocessed_2["dob"].dt.year
df_preprocessed_2["birth_month"] = df_preprocessed_2["dob"].dt.month
df_preprocessed_2["birth_day"] = df_preprocessed_2["dob"].dt.day

In [21]:
df_preprocessed_2.drop(["trans_date_trans_time", "dob"], axis=1, inplace = True)

In [22]:
#Select the new created cols
time_cols = ["transaction_year", "transaction_month", "transaction_day", "transaction_hour",\
    "transaction_minute", "transaction_second", "birth_year", "birth_month", "birth_day"]

df_preprocessed_2[time_cols].head(5)

,transaction_year,transaction_month,transaction_day,transaction_hour,transaction_minute,transaction_second,birth_year,birth_month,birth_day
0,2019,1,1,0,0,18,1988,3,9
1,2019,1,1,0,0,44,1978,6,21
2,2019,1,1,0,0,51,1962,1,19
3,2019,1,1,0,1,16,1967,1,12
4,2019,1,1,0,3,6,1986,3,28


### Preprocessing_3: Lets encode the gender and low categorical features:
1. Gender would be binary
2. One-hot-encoding for category and state columns (14 and 51 unique values)

In [23]:
# Lets set our new variables
df_preprocessed_3 = copy.copy(df_preprocessed_2)

#Now, lets do it for gender
df_preprocessed_3["gender"] = df_preprocessed_3["gender"].map({"F": 1, "M": 0})

In [24]:
print(df_preprocessed_3["gender"].head(5))

0    1
1    1
2    0
3    0
4    0
Name: gender, dtype: int64


In [25]:
#Lets now use pd.get_dummies for the other ones with low values
df_preprocessed_3 = pd.get_dummies(df_preprocessed_3, columns=['category', 'state'], drop_first=True, dtype=int)

In [26]:
print([x for x in df_preprocessed_3.columns])
print(df_preprocessed_3.columns.shape)

['cc_num', 'merchant', 'amt', 'first', 'last', 'gender', 'street', 'city', 'zip', 'lat', 'long', 'city_pop', 'job', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud', 'merch_zipcode', 'transaction_year', 'transaction_month', 'transaction_day', 'transaction_hour', 'transaction_minute', 'transaction_second', 'birth_year', 'birth_month', 'birth_day', 'category_food_dining', 'category_gas_transport', 'category_grocery_net', 'category_grocery_pos', 'category_health_fitness', 'category_home', 'category_kids_pets', 'category_misc_net', 'category_misc_pos', 'category_personal_care', 'category_shopping_net', 'category_shopping_pos', 'category_travel', 'state_AL', 'state_AR', 'state_AZ', 'state_CA', 'state_CO', 'state_CT', 'state_DC', 'state_DE', 'state_FL', 'state_GA', 'state_HI', 'state_IA', 'state_ID', 'state_IL', 'state_IN', 'state_KS', 'state_KY', 'state_LA', 'state_MA', 'state_MD', 'state_ME', 'state_MI', 'state_MN', 'state_MO', 'state_MS', 'state_MT', 'state_NC', 'state_ND', 'state_NE', 

### Lets take a look into the values
We are almost there; and in order to get a sense of the data. lets print some preprocessed values to see what kind of variables we have

In [27]:
#For loop for the pre-processed columns
for c in df_preprocessed_3.columns:
    print(df_preprocessed_3[c].head(3))
    print("\n")

0    2703186189652095
1        630423337322
2      38859492057661
Name: cc_num, dtype: int64


0         fraud_Rippin, Kub and Mann
1    fraud_Heller, Gutmann and Zieme
2               fraud_Lind-Buckridge
Name: merchant, dtype: object


0      4.97
1    107.23
2    220.11
Name: amt, dtype: float64


0     Jennifer
1    Stephanie
2       Edward
Name: first, dtype: object


0      Banks
1       Gill
2    Sanchez
Name: last, dtype: object


0    1
1    1
2    0
Name: gender, dtype: int64


0                  561 Perry Cove
1    43039 Riley Greens Suite 393
2        594 White Dale Suite 530
Name: street, dtype: object


0    Moravian Falls
1            Orient
2        Malad City
Name: city, dtype: object


0    28654
1    99160
2    83252
Name: zip, dtype: int64


0    36.0788
1    48.8878
2    42.1808
Name: lat, dtype: float64


0    -81.1781
1   -118.2105
2   -112.2620
Name: long, dtype: float64


0    3495
1     149
2    4154
Name: city_pop, dtype: int64


0            Psychologist, co

### Preprocessing_4: Frequency encoding and Target encoding for the rest categorical features
1. Determine the frequency for columns: merchant, first, last, Street, City, job
2. Use that to map for the rest of them
3. Using a TargetEncoder() instance, encode the merchant with target variable
4. Drop all original columns

In [28]:
#Lets see the columns for the names
col_names = ["merchant", "first", "last", "street", "city", "job"]

#Lets set the df_preprocessed_4
df_preprocessed_4 = copy.copy(df_preprocessed_3)

In [29]:
from category_encoders import TargetEncoder

encoder = TargetEncoder()

for c in col_names:
    #Create a name with _encoded and _freq
    col_enc_name = c +"_encoded"
    col_freq_name = c+ "_freq"

    #Use the target encoder to create the new column
    df_preprocessed_4[col_enc_name] = encoder.fit_transform(df_preprocessed_4[c], df_preprocessed_4["is_fraud"])

    #Obtain the frequency of the unique names for each
    freq = df_preprocessed_4[c].value_counts()
    df_preprocessed_4[col_freq_name] = df_preprocessed_4[c].map(freq)

    # Drop the original passed column
    df_preprocessed_4.drop(c, axis=1, inplace=True)

In [29]:
print(df_preprocessed_4.columns)

Index(['cc_num', 'amt', 'gender', 'zip', 'lat', 'long', 'city_pop',
       'unix_time', 'merch_lat', 'merch_long', 'is_fraud', 'merch_zipcode',
       'transaction_year', 'transaction_month', 'transaction_day',
       'transaction_hour', 'transaction_minute', 'transaction_second',
       'birth_year', 'birth_month', 'birth_day', 'category_food_dining',
       'category_gas_transport', 'category_grocery_net',
       'category_grocery_pos', 'category_health_fitness', 'category_home',
       'category_kids_pets', 'category_misc_net', 'category_misc_pos',
       'category_personal_care', 'category_shopping_net',
       'category_shopping_pos', 'category_travel', 'state_AL', 'state_AR',
       'state_AZ', 'state_CA', 'state_CO', 'state_CT', 'state_DC', 'state_DE',
       'state_FL', 'state_GA', 'state_HI', 'state_IA', 'state_ID', 'state_IL',
       'state_IN', 'state_KS', 'state_KY', 'state_LA', 'state_MA', 'state_MD',
       'state_ME', 'state_MI', 'state_MN', 'state_MO', 'state_MS', 'stat

In [30]:
print(df_preprocessed_4.shape)

(1296675, 96)


In [31]:
print(df_preprocessed_4.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 96 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   cc_num                   1296675 non-null  int64  
 1   amt                      1296675 non-null  float64
 2   gender                   1296675 non-null  int64  
 3   zip                      1296675 non-null  int64  
 4   lat                      1296675 non-null  float64
 5   long                     1296675 non-null  float64
 6   city_pop                 1296675 non-null  int64  
 7   unix_time                1296675 non-null  int64  
 8   merch_lat                1296675 non-null  float64
 9   merch_long               1296675 non-null  float64
 10  is_fraud                 1296675 non-null  int64  
 11  merch_zipcode            1296675 non-null  float64
 12  transaction_year         1296675 non-null  int32  
 13  transaction_month        1296675 non-null 

In [32]:
print(df_preprocessed_4.describe())

             cc_num           amt        gender           zip           lat  \
count  1.296675e+06  1.296675e+06  1.296675e+06  1.296675e+06  1.296675e+06   
mean   4.171920e+17  7.035104e+01  5.474487e-01  4.880067e+04  3.853762e+01   
std    1.308806e+18  1.603160e+02  4.977437e-01  2.689322e+04  5.075808e+00   
min    6.041621e+10  1.000000e+00  0.000000e+00  1.257000e+03  2.002710e+01   
25%    1.800429e+14  9.650000e+00  0.000000e+00  2.623700e+04  3.462050e+01   
50%    3.521417e+15  4.752000e+01  1.000000e+00  4.817400e+04  3.935430e+01   
75%    4.642255e+15  8.314000e+01  1.000000e+00  7.204200e+04  4.194040e+01   
max    4.992346e+18  2.894890e+04  1.000000e+00  9.978300e+04  6.669330e+01   

               long      city_pop     unix_time     merch_lat    merch_long  \
count  1.296675e+06  1.296675e+06  1.296675e+06  1.296675e+06  1.296675e+06   
mean  -9.022634e+01  8.882444e+04  1.349244e+09  3.853734e+01 -9.022646e+01   
std    1.375908e+01  3.019564e+05  1.284128e+07  5.